# Random Forest training
This is an auto-generated notebook. To reproduce these results, attach this notebook to the **ML Cluster** cluster and rerun it.
- Compare trials in the [MLflow experiment](#mlflow/experiments/3508490397944312/s?orderByKey=metrics.%60val_f1_score%60&orderByAsc=false)
- Navigate to the parent notebook [here](#notebook/3508490397944302) (If you launched the AutoML experiment using the Experiments UI, this link isn't very useful.)
- Clone this notebook into your project folder by selecting **File > Clone** in the notebook toolbar.

Runtime Version: _10.4.x-cpu-ml-scala2.12_

In [0]:
import mlflow
import databricks.automl_runtime

target_col = "deceptive"

## Load Data

In [0]:
from mlflow.tracking import MlflowClient
import os
import uuid
import shutil
import pandas as pd

# Create temp directory to download input data from MLflow
input_temp_dir = os.path.join(os.environ["SPARK_LOCAL_DIRS"], "tmp", str(uuid.uuid4())[:8])
os.makedirs(input_temp_dir)


# Download the artifact and read it into a pandas DataFrame
input_client = MlflowClient()
input_data_path = input_client.download_artifacts("e2d1e92b0b564b439e1e8d6849cdf45e", "data", input_temp_dir)

df_loaded = pd.read_parquet(os.path.join(input_data_path, "training_data"))
# Delete the temp data
shutil.rmtree(input_temp_dir)

# Preview data
df_loaded.head(5)

Out[2]:

,hotel,polarity,source,text,deceptive
0,conrad,negative,Web,Booked a room w/ a queen bed for 2 nights for ...,truthful
1,hyatt,negative,Web,I arrived and my friends were already there an...,truthful
2,hyatt,negative,Web,This was the 2nd time that we have stayed at t...,truthful
3,omni,negative,Web,My daughter and I woke in the morning wanting ...,truthful
4,hyatt,negative,Web,They allowed parties to go on all night and bo...,truthful


### Select supported columns
Select only the columns that are supported. This allows us to train a model that can predict on a dataset that has extra columns that are not used in training.
`[]` are dropped in the pipelines. See the Alerts tab of the AutoML Experiment page for details on why these columns are dropped.

In [0]:
from databricks.automl_runtime.sklearn.column_selector import ColumnSelector
supported_cols = ["hotel", "source", "polarity", "text"]
col_selector = ColumnSelector(supported_cols)

## Preprocessors

In [0]:
transformers = []

### Categorical columns

#### Low-cardinality categoricals
Convert each low-cardinality categorical column into multiple binary columns through one-hot encoding.
For each input categorical column (string or numeric), the number of output columns is equal to the number of unique values in the input column.

In [0]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

one_hot_imputers = []

one_hot_pipeline = Pipeline(steps=[
    ("imputers", ColumnTransformer(one_hot_imputers, sparse_threshold=0, remainder="passthrough")),
    ("one_hot_encoder", OneHotEncoder(handle_unknown="ignore")),
])

transformers.append(("onehot", one_hot_pipeline, ["hotel", "polarity", "source"]))

### Text features
Convert each feature to a fixed-length vector using TF-IDF vectorization. The length of the output
vector is equal to 1024. Each column corresponds to one of the top word n-grams
where n is in the range [1, 2].

In [0]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer

for col in {'text'}:
    vectorizer = Pipeline(steps=[
        ("imputer", SimpleImputer(missing_values=None, strategy="constant", fill_value="")),
        # Reshape to 1D since SimpleImputer changes the shape of the input to 2D
        ("reshape", FunctionTransformer(np.reshape, kw_args={"newshape":-1})),
        ("tfidf", TfidfVectorizer(decode_error="ignore", ngram_range = (1, 2), max_features=1024))])

    transformers.append((f"text_{col}", vectorizer, [col]))

In [0]:
from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer(transformers, remainder="passthrough", sparse_threshold=0)

### Feature standardization
Scale all feature columns to be centered around zero with unit variance.

In [0]:
from sklearn.preprocessing import StandardScaler

standardizer = StandardScaler()

## Train - Validation - Test Split
Split the input data into 3 sets:
- Train (60% of the dataset used to train the model)
- Validation (20% of the dataset used to tune the hyperparameters of the model)
- Test (20% of the dataset used to report the true performance of the model on an unseen dataset)

In [0]:
from sklearn.model_selection import train_test_split

split_X = df_loaded.drop([target_col], axis=1)
split_y = df_loaded[target_col]

# Split out train data
X_train, split_X_rem, y_train, split_y_rem = train_test_split(split_X, split_y, train_size=0.6, random_state=151528212, stratify=split_y)

# Split remaining data equally for validation and test
X_val, X_test, y_val, y_test = train_test_split(split_X_rem, split_y_rem, test_size=0.5, random_state=151528212, stratify=split_y_rem)

## Train classification model
- Log relevant metrics to MLflow to track runs
- All the runs are logged under [this MLflow experiment](#mlflow/experiments/3508490397944312/s?orderByKey=metrics.%60val_f1_score%60&orderByAsc=false)
- Change the model parameters and re-run the training cell to log a different trial to the MLflow experiment
- To view the full list of tunable hyperparameters, check the output of the cell below

In [0]:
from sklearn.ensemble import RandomForestClassifier

help(RandomForestClassifier)

Help on class RandomForestClassifier in module sklearn.ensemble._forest:

class RandomForestClassifier(ForestClassifier)
 RandomForestClassifier(n_estimators=100, *, criterion='gini', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='auto', max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, bootstrap=True, oob_score=False, n_jobs=None, random_state=None, verbose=0, warm_start=False, class_weight=None, ccp_alpha=0.0, max_samples=None)
 
 A random forest classifier.
 
 A random forest is a meta estimator that fits a number of decision tree
 classifiers on various sub-samples of the dataset and uses averaging to
 improve the predictive accuracy and control over-fitting.
 The sub-sample size is controlled with the `max_samples` parameter if
 `bootstrap=True` (default), otherwise the whole dataset is used to build
 each tree.
 
 Read more in the :ref:`User Guide <forest>`.
 
 Parameters
 ----------
 n_estimators : int, default=100
 The number of trees in the forest.
 
 .. versionchanged:: 0.22
 The default value of ``n_estimators`` changed from 10 to 100
 in 0.22.
 
 criterion : {"gini", "entropy"}, default="gini"
 The function to measure the quality of a split. Supported criteria are
 "gini" for the Gini impurity and "entropy" for the information gain.
 Note: this parameter is tree-specific.
 
 max_depth : int, default=None
 The maximum depth of the tree. If None, then nodes are expanded until
 all leaves are pure or until all leaves contain less than
 min_samples_split samples.
 
 min_samples_split : int or float, default=2
 The minimum number of samples required to split an internal node:
 
 - If int, then consider `min_samples_split` as the minimum number.
 - If float, then `min_samples_split` is a fraction and
 `ceil(min_samples_split * n_samples)` are the minimum
 number of samples for each split.
 
 .. versionchanged:: 0.18
 Added float values for fractions.
 
 min_samples_leaf : int or float, default=1
 The minimum number of samples required to be at a leaf node.
 A split point at any depth will only be considered if it leaves at
 least ``min_samples_leaf`` training samples in each of the left and
 right branches. This may have the effect of smoothing the model,
 especially in regression.
 
 - If int, then consider `min_samples_leaf` as the minimum number.
 - If float, then `min_samples_leaf` is a fraction and
 `ceil(min_samples_leaf * n_samples)` are the minimum
 number of samples for each node.
 
 .. versionchanged:: 0.18
 Added float values for fractions.
 
 min_weight_fraction_leaf : float, default=0.0
 The minimum weighted fraction of the sum total of weights (of all
 the input samples) required to be at a leaf node. Samples have
 equal weight when sample_weight is not provided.
 
 max_features : {"auto", "sqrt", "log2"}, int or float, default="auto"
 The number of features to consider when looking for the best split:
 
 - If int, then consider `max_features` features at each split.
 - If float, then `max_features` is a fraction and
 `round(max_features * n_features)` features are considered at each
 split.
 - If "auto", then `max_features=sqrt(n_features)`.
 - If "sqrt", then `max_features=sqrt(n_features)` (same as "auto").
 - If "log2", then `max_features=log2(n_features)`.
 - If None, then `max_features=n_features`.
 
 Note: the search for a split does not stop until at least one
 valid partition of the node samples is found, even if it requires to
 effectively inspect more than ``max_features`` features.
 
 max_leaf_nodes : int, default=None
 Grow trees with ``max_leaf_nodes`` in best-first fashion.
 Best nodes are defined as relative reduction in impurity.
 If None then unlimited number of leaf nodes.
 
 min_impurity_decrease : float, default=0.0
 A node will be split if this split induces a decrease of the impurity
 greater than or equal to this value.
 
 The weighted impurity decrease equation is the following::
 
 N_t / N * (impurity - N_

In [0]:
import mlflow
import sklearn
from sklearn import set_config
from sklearn.pipeline import Pipeline

set_config(display="diagram")

skrf_classifier = RandomForestClassifier(
  bootstrap=False,
  criterion="gini",
  max_depth=8,
  max_features=0.710683625096309,
  min_samples_leaf=0.17537208144585273,
  min_samples_split=0.16191668799356604,
  n_estimators=2299,
  random_state=151528212,
)

model = Pipeline([
    ("column_selector", col_selector),
    ("preprocessor", preprocessor),
    ("standardizer", standardizer),
    ("classifier", skrf_classifier),
])

model

Out[11]:

Pipeline Pipeline(steps=[('column_selector',
 ColumnSelector(cols=['hotel', 'source', 'polarity', 'text'])),
 ('preprocessor',
 ColumnTransformer(remainder='passthrough', sparse_threshold=0,
 transformers=[('onehot',
 Pipeline(steps=[('imputers',
 ColumnTransformer(remainder='passthrough',
 sparse_threshold=0,
 transformers=[])),
 ('one_hot_encoder',
 OneHotEncoder(handle_unknown=...
 kw_args={'newshape': -1})),
 ('tfidf',
 TfidfVectorizer(decode_error='ignore',
 max_features=1024,
 ngram_range=(1,
 2)))]),
 ['text'])])),
 ('standardizer', StandardScaler()),
 ('classifier',
 RandomForestClassifier(bootstrap=False, max_depth=8,
 max_features=0.710683625096309,
 min_samples_leaf=0.17537208144585273,
 min_samples_split=0.16191668799356604,
 n_estimators=2299,
 random_state=151528212))]) ColumnSelector ColumnSelector(cols=['hotel', 'source', 'polarity', 'text']) preprocessor: ColumnTransformer ColumnTransformer(remainder='passthrough', sparse_threshold=0,
 transformers=[('onehot',
 Pipeline(steps=[('imputers',
 ColumnTransformer(remainder='passthrough',
 sparse_threshold=0,
 transformers=[])),
 ('one_hot_encoder',
 OneHotEncoder(handle_unknown='ignore'))]),
 ['hotel', 'polarity', 'source']),
 ('text_text',
 Pipeline(steps=[('imputer',
 SimpleImputer(fill_value='',
 missing_values=None,
 strategy='constant')),
 ('reshape',
 FunctionTransformer(func= ,
 kw_args={'newshape': -1})),
 ('tfidf',
 TfidfVectorizer(decode_error='ignore',
 max_features=1024,
 ngram_range=(1,
 2)))]),
 ['text'])]) onehot ['hotel', 'polarity', 'source'] imputers: ColumnTransformer ColumnTransformer(remainder='passthrough', sparse_threshold=0, transformers=[]) remainder passthrough passthrough OneHotEncoder OneHotEncoder(handle_unknown='ignore') text_text ['text'] SimpleImputer SimpleImputer(fill_value='', missing_values=None, strategy='constant') FunctionTransformer FunctionTransformer(func= ,
 kw_args={'newshape': -1}) TfidfVectorizer TfidfVectorizer(decode_error='ignore', max_features=1024, ngram_range=(1, 2)) remainder passthrough passthrough StandardScaler StandardScaler() RandomForestClassifier RandomForestClassifier(bootstrap=False, max_depth=8,
 max_features=0.710683625096309,
 min_samples_leaf=0.17537208144585273,
 min_samples_split=0.16191668799356604, n_estimators=2299,
 random_state=151528212)

In [0]:
# Enable automatic logging of input samples, metrics, parameters, and models
mlflow.sklearn.autolog(log_input_examples=True, silent=True)

with mlflow.start_run(experiment_id="3508490397944312", run_name="random_forest") as mlflow_run:
    model.fit(X_train, y_train)
    
    # Training metrics are logged by MLflow autologging
    # Log metrics for the validation set
    skrf_val_metrics = mlflow.sklearn.eval_and_log_metrics(model, X_val, y_val, prefix="val_")

    # Log metrics for the test set
    skrf_test_metrics = mlflow.sklearn.eval_and_log_metrics(model, X_test, y_test, prefix="test_")

    # Display the logged metrics
    skrf_val_metrics = {k.replace("val_", ""): v for k, v in skrf_val_metrics.items()}
    skrf_test_metrics = {k.replace("test_", ""): v for k, v in skrf_test_metrics.items()}
    display(pd.DataFrame([skrf_val_metrics, skrf_test_metrics], index=["validation", "test"]))

precision_score,recall_score,f1_score,accuracy_score,log_loss,roc_auc_score,score
1.0,1.0,1.0,1.0,0.006333888264696429,0.9999999999999999,1.0
1.0,1.0,1.0,1.0,0.006443845029823922,1.0,1.0


## Feature importance

SHAP is a game-theoretic approach to explain machine learning models, providing a summary plot
of the relationship between features and model output. Features are ranked in descending order of
importance, and impact/color describe the correlation between the feature and the target variable.
- Generating SHAP feature importance is a very memory intensive operation, so to ensure that AutoML can run trials without
  running out of memory, we disable SHAP by default.<br />
  You can set the flag defined below to `shap_enabled = True` and re-run this notebook to see the SHAP plots.
- To reduce the computational overhead of each trial, a single example is sampled from the validation set to explain.<br />
  For more thorough results, increase the sample size of explanations, or provide your own examples to explain.
- SHAP cannot explain models using data with nulls; if your dataset has any, both the background data and
  examples to explain will be imputed using the mode (most frequent values). This affects the computed
  SHAP values, as the imputed samples may not match the actual data distribution.

For more information on how to read Shapley values, see the [SHAP documentation](https://shap.readthedocs.io/en/latest/example_notebooks/overviews/An%20introduction%20to%20explainable%20AI%20with%20Shapley%20values.html).

In [0]:
# Set this flag to True and re-run the notebook to see the SHAP plots
shap_enabled = False

In [0]:
if shap_enabled:
    from shap import KernelExplainer, summary_plot
    # Sample background data for SHAP Explainer. Increase the sample size to reduce variance.
    train_sample = X_train.sample(n=min(100, len(X_train.index)))

    # Sample a single example from the validation set to explain. Increase the sample size and rerun for more thorough results.
    example = X_val.sample(n=1)

    # Use Kernel SHAP to explain feature importance on the example from the validation set.
    predict = lambda x: model.predict_proba(pd.DataFrame(x, columns=X_train.columns))
    explainer = KernelExplainer(predict, train_sample, link="logit")
    shap_values = explainer.shap_values(example, l1_reg=False)
    summary_plot(shap_values, example, class_names=model.classes_)

## Inference
[The MLflow Model Registry](https://docs.databricks.com/applications/mlflow/model-registry.html) is a collaborative hub where teams can share ML models, work together from experimentation to online testing and production, integrate with approval and governance workflows, and monitor ML deployments and their performance. The snippets below show how to add the model trained in this notebook to the model registry and to retrieve it later for inference.

> **NOTE:** The `model_uri` for the model already trained in this notebook can be found in the cell below

### Register to Model Registry
```
model_name = "Example"

model_uri = f"runs:/{ mlflow_run.info.run_id }/model"
registered_model_version = mlflow.register_model(model_uri, model_name)
```

### Load from Model Registry
```
model_name = "Example"
model_version = registered_model_version.version

model = mlflow.pyfunc.load_model(model_uri=f"models:/{model_name}/{model_version}")
model.predict(input_X)
```

### Load model without registering
```
model_uri = f"runs:/{ mlflow_run.info.run_id }/model"

model = mlflow.pyfunc.load_model(model_uri)
model.predict(input_X)
```

In [0]:
# model_uri for the generated model
print(f"runs:/{ mlflow_run.info.run_id }/model")

runs:/9b4ca0ad248a448facd50107fbb57b71/model